In [1]:
import cv2
import os, sys
import time
from moviepy.video.io.VideoFileClip import VideoFileClip
from pathlib import Path

In [2]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class TextStyle:
    start = "\033[1m"
    end = "\033[0;0m"

In [3]:
def generate_output_string():
    from datetime import datetime

    now = datetime.now()
    output_str = now.strftime("%d_%m_%Y_%H_%M_%S") + "_output"
    return output_str

In [4]:
class ExtractVideoSegment:
    def __init__(self, video_path, write_path='./'):
        self.video_path = video_path
        self.write_path = write_path
        self.filename = Path(self.video_path).stem
        self.extension = Path(self.video_path).suffix
        self.start_frame = None
        self.end_frame = None
        self.current_frame = 0
        self.video_index = 0
        self.cap, self.num_frames, self.fps = self._get_video_capture_object()
        
        cv2.namedWindow('preview', cv2.WINDOW_NORMAL)
        cv2.createTrackbar('scroll', 'preview', 0, self.num_frames, self._trackbar_callback)

        self.font = cv2.FONT_HERSHEY_DUPLEX
        
        self.instructions = [
            "Press 's' to start cutting",
            " ",
            "Press 'e' to end cutting",
            " ",
            "Press 'c' to confirm",
            " ",
            "Press 'Esc' to exit",
        ]

    def _get_video_capture_object(self):
        try:
            cap = cv2.VideoCapture(self.video_path)

            if cap.isOpened():
                num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                fps = int(cap.get(cv2.CAP_PROP_FPS))

                if num_frames > 0:
                    print("==================")
                    print(f"#Frames: {num_frames}")
                    print(f"Video FPS: {fps}")
                    print("==================")
                    print()
                else:
                    print("The video file is empty or there was an issue reading the frame count.")
                    sys.exit(0)
            else:
                print("The video could not be opened.")

        except cv2.error as e:
            print(f"OpenCV error: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")
        # finally:
            # video.release()
        return cap, num_frames, fps

    def _trackbar_callback(self, x):
        self.current_frame = x
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, self.current_frame)
        self._display_current_frame()

    def _get_formatted_time(self, seconds):
        return time.strftime('%H:%M:%S', time.gmtime(seconds))

    def _extract_and_save_segment(self, start_frame, end_frame, output_filename):
        start_time = start_frame / self.fps
        duration = (end_frame - start_frame) / self.fps

        clip = VideoFileClip(self.video_path).subclip(start_time, start_time + duration)
        clip.write_videofile(output_filename, codec='libx264', audio_codec='aac')

    def _display_current_frame(self):
        ret, frame = self.cap.read()
        if ret:
            frame_with_instructions = frame.copy()
            height, width, _ = frame_with_instructions.shape

            # Determine the dimensions of the red background based on the instructions
            text_size = cv2.getTextSize(max(self.instructions, key=len), self.font, 1, 2)[0]
            padding = 10
            bg_height = len(self.instructions) * text_size[1] + padding * 2
            bg_width = text_size[0] + padding * 2

            # Draw the red background behind the instructions
            cv2.rectangle(frame_with_instructions, (width - bg_width, 0), (width, bg_height), (0, 0, 255), -1)

            # Display instructions on the red background
            for idx, instruction in enumerate(self.instructions):
                y = (idx + 1) * text_size[1] + padding
                cv2.putText(frame_with_instructions, instruction, (width - bg_width + padding, y), self.font, 1, (255, 255, 255), 2)
            
            # Display the frame
            cv2.imshow('preview', frame_with_instructions)

    def cut_video(self):
        self._display_current_frame()  # Display initial frame
        while True:
            k = cv2.waitKey(1)

            if k == 83 or k == 115:  # Press 's' key
                self.start_frame = self.current_frame
                start_time = self._get_formatted_time(self.start_frame / self.fps)
                print()
                print()
                print(f"{bcolors.OKGREEN} {TextStyle.start}[Segment Info]: {TextStyle.end} {bcolors.ENDC}")
                print(f"Start Frame: {self.start_frame}")
                print(f"Start Time: {start_time}")


            if k == 69 or k == 101:  # Press 'e' key
                self.end_frame = self.current_frame
                end_time = self._get_formatted_time(self.end_frame / self.fps)
                print(f"End Frame: {self.end_frame}")
                print(f"End Time: {end_time}")

            if k == 67 or k == 99:    # Press 'c' key
                self.video_index += 1
                # output_filename = os.path.join(self.write_path, f'out_{self.video_index}.mp4')
                output_str = self.filename + "_" + generate_output_string()
                output_filename = os.path.join(self.write_path, output_str+self.extension)

                if self.start_frame is not None and self.end_frame is not None:
                    print(f"Saving segment from frame {self.start_frame} to {self.end_frame}...")
                    self._extract_and_save_segment(self.start_frame, self.end_frame, output_filename)
                else:
                    print("Please set start and end frames before confirming.")

                self.start_frame = None
                self.end_frame = None
                

            if k == 27:  # Press 'Esc' key
                break

        cv2.destroyAllWindows()
        self.cap.release()


In [5]:
if __name__ == "__main__":
    abs_video_path = '/home/acer/workspace/CV-Local/unfactored-code/MVI_20032.mp4'
    video_cutter = ExtractVideoSegment(abs_video_path)
    video_cutter.cut_video()

#Frames: 437
Video FPS: 10



 [Segment Info]:  
Start Frame: 0
Start Time: 00:00:00
End Frame: 8
End Time: 00:00:00
Saving segment from frame 0 to 8...
Moviepy - Building video ./MVI_20032_29_10_2023_13_19_54_output.mp4.
Moviepy - Writing video ./MVI_20032_29_10_2023_13_19_54_output.mp4



Moviepy - Done !
Moviepy - video ready ./MVI_20032_29_10_2023_13_19_54_output.mp4




 [Segment Info]:  
Start Frame: 12
Start Time: 00:00:01
End Frame: 31
End Time: 00:00:03
Saving segment from frame 12 to 31...
Moviepy - Building video ./MVI_20032_29_10_2023_13_20_02_output.mp4.
Moviepy - Writing video ./MVI_20032_29_10_2023_13_20_02_output.mp4



Moviepy - Done !
Moviepy - video ready ./MVI_20032_29_10_2023_13_20_02_output.mp4


End Frame: 40
End Time: 00:00:04


 [Segment Info]:  
Start Frame: 40
Start Time: 00:00:04
End Frame: 70
End Time: 00:00:07
Saving segment from frame 40 to 70...
Moviepy - Building video ./MVI_20032_29_10_2023_13_20_09_output.mp4.
Moviepy - Writing video ./MVI_20032_29_10_2023_13_20_09_output.mp4



Moviepy - Done !
Moviepy - video ready ./MVI_20032_29_10_2023_13_20_09_output.mp4


 [Segment Info]:  
Start Frame: 79
Start Time: 00:00:07
End Frame: 128
End Time: 00:00:12
Saving segment from frame 79 to 128...
Moviepy - Building video ./MVI_20032_29_10_2023_13_20_19_output.mp4.
Moviepy - Writing video ./MVI_20032_29_10_2023_13_20_19_output.mp4



Moviepy - Done !
Moviepy - video ready ./MVI_20032_29_10_2023_13_20_19_output.mp4
